<a href="https://colab.research.google.com/github/AUT-Student/CN-HW2/blob/main/CN_HW2_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: Complex Network
<br>
<b>Description:</b> HomeWork 2 | Question 4
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Libraries

In [8]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn
import matplotlib.pyplot as plt

In [6]:
!pip install --upgrade scipy networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 36.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3


# Dataset

In [2]:
!gdown https://snap.stanford.edu/data/wiki-Vote.txt.gz

Downloading...
From: https://snap.stanford.edu/data/wiki-Vote.txt.gz
To: /content/wiki-Vote.txt.gz
100% 290k/290k [00:01<00:00, 241kB/s]


In [4]:
!gunzip /content/wiki-Vote.txt.gz

In [5]:
!head /content/wiki-Vote.txt 

# Directed graph (each unordered pair of nodes is saved once): Wiki-Vote.txt 
# Wikipedia voting on promotion to administratorship (till January 2008). Directed edge A->B means user A voted on B becoming Wikipedia administrator.
# Nodes: 7115 Edges: 103689
# FromNodeId	ToNodeId
30	1412
30	3352
30	5254
30	5543
30	7478
3	28


In [14]:
graph = nx.Graph()

with open("/content/wiki-Vote.txt") as file:
  for i, line in enumerate(file.readlines()[4:]):
    source, destination = line.split()[:2]
    graph.add_edge(source, destination)

# Algorithm

In [53]:
def modularity_matrix(graph):
  n = graph.number_of_nodes()
  m = graph.number_of_edges()

  A = nx.adjacency_matrix(graph).todense()
  B = np.zeros((n,n))

  node_degree = dict()
  for node in graph.nodes():
    node_degree[node] = graph.degree(node)

  for i, node_i in enumerate(graph.nodes()):
    for j, node_j in enumerate(graph.nodes()):
      B[i, j] = A[i, j] - (node_degree[node_i]*node_degree[node_j])/(2*m)
  
  return B

In [59]:
def largest_aigen_vector(matrix):
  values, vectors = np.linalg.eig(matrix)

  idx = values.argsort()[::-1]   
  values = values[idx]
  vectors = vectors[:,idx]

  return vectors[0] 

In [55]:
def modularity(graph, clusters):
  m = graph.number_of_edges()
  output = 0
  for cluster in set(clusters):
    cluster_nodes = []
    for i, node in enumerate(graph.nodes()):
      if cluster == clusters[i]:
        cluster_nodes.append(node)

    for node_i in cluster_nodes:
      for node_j in cluster_nodes:
        Aij = int(graph.has_edge(node_i, node_j))
        ki = graph.degree(node_i)
        kj = graph.degree(node_j)
        output += Aij - (ki*kj)/(2*m)
  
  output/=2*m
  return output

In [75]:
def split_graph(graph, clustering_vector):
  node_list1 = []
  node_list2 = []
  for i, node in enumerate(graph.nodes):
    if clustering_vector[i]==1:
      node_list1.append(node)
    else:
      node_list2.append(node)

  subgraph1 = graph.subgraph(node_list1)
  subgraph2 = graph.subgraph(node_list2)
  return subgraph1, subgraph2

In [94]:
def modularity_optimization(graph):
    # first_modularity = 0 # modularity(graph, np.zeros(graph.number_of_nodes()))
    B = modularity_matrix(graph)
    la_vector = largest_aigen_vector(B)
    clustering_vector = [float(x) for x in np.sign(la_vector)]
    new_modularity = modularity(graph, clustering_vector)

    if new_modularity>1e-6:
      subgraph1, subgraph2 = split_graph(graph, clustering_vector)

      print(f"Graph with {graph.number_of_nodes()} nodes splited to " +
            f"graphs with {subgraph1.number_of_nodes()} and {subgraph2.number_of_nodes()} nodes" + 
            f"\nIncrease modularity amount = {new_modularity}\n")

      subcommunities1 = modularity_optimization(subgraph1)
      
      subcommunities2 = modularity_optimization(subgraph2)

      return [*subcommunities1, *subcommunities2]

    else:

      print(f"Graph with {graph.number_of_nodes()} nodes Dosen't splited!" + 
            f"\nIncrease modularity amount = {new_modularity}\n")

      return [graph]

# Results

In [95]:
communities = modularity_optimization(graph)

<ipython-input-94-ef16592f97c4>:5: ComplexWarning: Casting complex values to real discards the imaginary part
  clustering_vector = [float(x) for x in np.sign(la_vector)]


Graph with 7115 nodes splited to graphs with 3548 and 3567 nodes
Increase modularity amount = 0.00012884827079628916

Graph with 3548 nodes Dosen't splited!
Increase modularity amount = -0.0002122656049376358

Graph with 3567 nodes Dosen't splited!
Increase modularity amount = -0.00646544354754817



In [102]:
communities[0].nodes()

NodeView(('7365', '2567', '670', '1681', '5118', '8293', '6193', '3305', '2316', '3801', '694', '4252', '945', '6717', '7349', '8229', '2029', '7875', '7833', '4191', '1275', '7446', '5120', '6535', '171', '2155', '3708', '8066', '8232', '1193', '3731', '4818', '4887', '7468', '1250', '7688', '6501', '4345', '5990', '258', '1649', '276', '1665', '4665', '2257', '6248', '4874', '3567', '4500', '2374', '7629', '1313', '1720', '7138', '1325', '6286', '5158', '5195', '5176', '7577', '4447', '7743', '3652', '7360', '2830', '6634', '7101', '3998', '5164', '4400', '457', '4844', '756', '7676', '5265', '4918', '2119', '1418', '4131', '6795', '594', '4016', '6929', '6149', '5414', '6678', '7300', '3298', '4547', '6862', '4553', '1966', '5748', '3294', '3633', '7850', '2868', '6020', '859', '6337', '6332', '81', '2957', '2894', '7392', '7119', '4944', '1236', '219', '677', '3750', '1809', '4496', '5351', '5653', '7869', '5655', '5932', '1213', '3352', '6472', '2351', '8135', '5412', '8194', '118

In [103]:
communities[1].nodes()

NodeView(('30', '5543', '7478', '178', '286', '348', '349', '567', '581', '604', '611', '8283', '6', '8', '19', '23', '29', '33', '35', '50', '86', '252', '261', '306', '310', '322', '325', '339', '377', '415', '439', '488', '514', '545', '549', '560', '650', '659', '664', '667', '673', '675', '696', '739', '857', '959', '994', '3755', '8282', '8284', '4', '56', '130', '147', '151', '341', '606', '61', '89', '163', '167', '168', '175', '34', '132', '135', '216', '227', '228', '236', '250', '274', '304', '319', '359', '362', '378', '392', '394', '406', '427', '431', '432', '433', '440', '447', '538', '566', '609', '619', '643', '668', '741', '761', '765', '779', '789', '791', '798', '820', '826', '871', '885', '913', '922', '932', '936', '955', '960', '978', '991', '1022', '1156', '1160', '1167', '1220', '1221', '1222', '1241', '1389', '1390', '1396', '1428', '1441', '1442', '1446', '1453', '1464', '1478', '1484', '1485', '1489', '1490', '1497', '1507', '1514', '1518', '1524', '1532', '